In [1]:
import ollama
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import OllamaEmbeddings

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [30]:
url = "https://github.com/paradite/sg-tech-list"
loader = WebBaseLoader(url)
docs = loader.load()
print(docs)
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=10)
splits = splitter.split_documents(docs)

[Document(page_content='\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nGitHub - paradite/sg-tech-list: :scroll: List of notable tech companies in Singapore\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSkip to content\n\n\n\n\n\n\n\n\n\n\n\n\nNavigation Menu\n\nToggle navigation\n\n\n\n\n\n\n\n\n\n\n          Sign in\n        \n\n\n \n\n\n\n\n\n\n\n\n\n\n\n\n\n        Product\n        \n\n\n\n\n\n\n\n\n\n\n\n\nActions\n        Automate any workflow\n      \n\n\n\n\n\n\n\nPackages\n        Host and manage packages\n      \n\n\n\n\n\n\n\nSecurity\n        Find and fix vulnerabilities\n      \n\n\n\n\n\n\n\nCodespaces\n        Instant dev environments\n      \n\n\n\n\n\n\n\nGitHub Copilot\n        Write better code with AI\n      \n\n\n\n\n\n\n\nCode review\n        Manage code changes\n      \n\n\n\n\n\n\n\nIssues\n        Plan and track work\n      \n\n\n\n\n\n

In [32]:
print('pulling model')
ollama.pull('llama3')
print('model pulled')
embedding_fn = OllamaEmbeddings(model='llama3')
print('creating vectors')
Chroma.from_documents(documents=splits, embedding=embedding_fn , persist_directory='./data/vectorstore')

pulling model
model pulled
creating vectors


In [2]:
def infer(prompt):
    req = [{'role': 'user', 'content': prompt}]
    resp = ollama.chat(model='llama3', messages=req)
    return resp['message']['content']

In [3]:
vectorstore = Chroma(persist_directory='./data/vectorstore', embedding_function=embedding_fn)
retriever = vectorstore.as_retriever()
def infer_with_context(question):
    docs = retriever.invoke(question)
    ctx = '\n\n'.join(doc.page_content for doc in docs)
    prompt = f"""
You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.

Question: {question} 

Context: {ctx} 

Answer:
"""
    return infer(prompt)

NameError: name 'embedding_fn' is not defined

In [35]:
qn = "Which Crypto companies in Singapore experienced layoffs?"
print(infer_with_context(qn))

Based on the provided context, the Crypto companies in Singapore that experienced layoffs are:

* Binance: Layoff 1000 in July 2023
* Cake DeFi: Layoff 34 since 2023
* Nansen: Layoff 30%

Note: I don't know about other companies not listed here.


In [36]:
qn = "Which companies are actively hiring?"
print(infer_with_context(qn))

According to the provided context, some companies that are actively hiring (or had recent layoffs) include: Rakuten Viki, Agoda, Shopify, GitLab, Salesforce, Twilio, Coupang, TheTradeDesk, ExpressVPN, and VMware.
